## README

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import json
import os
import time
import requests

from huggingface_hub import login

from src.data import extract
from src.tools.startup import settings
from src.agent import question_answering

## Parameters

In [ ]:
processed_folder = settings["volumes"]["processed"]
answers_file_path = os.path.join(processed_folder, "answers.json")

In [ ]:
# HuggingFace GAIA questions API
hf_base_url = "https://agents-course-unit4-scoring.hf.space"
hf_url_questions = f"{base_url}/questions"
hf_url_files = f"{base_url}/files/"
hf_url_submit = f"{base_url}/submit"
hf_space_url = "https://huggingface.co/spaces/casals90/AgentsCourseFinalAssignment/tree/main"

interim_folder = settings["volumes"]["interim"]

graph_config = {
    "configurable": {
        "thread_id": "1"}, 
    "recursion_limit": 30
}

Logging to Hugginface 

In [ ]:
login(os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
settings

## Extract

In [ ]:
questions = extract.get_questions(interim_folder)

In [ ]:
answers_list = extract.read_json_file(answers_file_path)
correct = extract.read_json_file("correct.json")

In [ ]:
answers = []
for i, question in enumerate(questions, start=1):
    task_id = question["task_id"]
    if task_id in ("8e867cd7-cff9-4e6c-867a-ff5ddc2550be", "a1e91b78-d3d8-4675-bb8d-62741b4b68a6", 
                   "9d191bce-651d-4746-be2d-7ef8ecadb9c2", "7bd855d8-463d-4ed5-93ca-5fe35145f733", 
                   "a0c07678-e491-4bbc-8f0b-07405144218f"):
        continue

    if task_id not in correct["correct"]:
        print(f"Question {i}: {question['question']}")
        print(question['task_id'])
        print("*"*30)

## Answer questions

In [ ]:
qa_agent = question_answering.QuestionAnsweringAgent(graph_config)

In [ ]:
#answer = qa_agent.answer_question(
#    "1+1 = 2?", stream_mode="values", subgraphs=False, debug=False)

In [ ]:
# answer

In [ ]:
#question = questions[3]

#query = question["question"]
#question

# answer = qa_agent.answer_gaia_question(
#    question, stream_mode="values", subgraphs=False, debug=False)

In [ ]:
answers = []
for i, question in enumerate(questions, start=1):
    task_id = question["task_id"]
    if task_id in ("8e867cd7-cff9-4e6c-867a-ff5ddc2550be", "a1e91b78-d3d8-4675-bb8d-62741b4b68a6", 
                   "9d191bce-651d-4746-be2d-7ef8ecadb9c2", "7bd855d8-463d-4ed5-93ca-5fe35145f733", 
                   "a0c07678-e491-4bbc-8f0b-07405144218f"):
        continue
    
    query = question["question"]
    base64_image = ""

    if task_id not in correct["correct"]:
        print(f"Question {i}: {question['question']}")
        print("*"*30)
        
        answer = qa_agent.answer_gaia_question(
            question, stream_mode="values", subgraphs=False, debug=False)
        
    else:
        for ans in answers_list:
            if ans["task_id"] == task_id:
                answer = ans["submitted_answer"]
                break


    answers.append({
      "task_id": task_id,
      "submitted_answer": answer
    })

In [ ]:
answers

submit_data = {
  "username": "casals90",
  "agent_code": space_url,
  "answers": answers
}

response = requests.post(url_submit, data=json.dumps(submit_data))

response.text

In [ ]:
response = load.submit_answers(answers)


In [ ]:
correct = []
for i in range(1, len(answers) + 1):
    iter_answer = answers[i-1:i] 
    task_id = iter_answer[0]["task_id"]

    submit_data = {
      "username": "casals90",
      "agent_code": space_url,
      "answers": iter_answer
    }
    time.sleep(5)
    response = requests.post(url_submit, data=json.dumps(submit_data))
    res = json.loads(response.text)
    if res["correct_count"] == 1:
        correct.append(task_id)

In [ ]:
correct

In [ ]:
correct_task_ids = {
    "correct": correct
}

correct_task_ids

In [ ]:
from src.data import load

# load.save_json_file(correct_task_ids, "correct.json")

In [ ]:
#with open("answers_75_score.json", "w") as f:
#    f.write(json.dumps(answers))